## Large language model project using Retrieval Augmented Generation (RAG) to provide more contextual, up to date results

### The dataset (source: Kaggle.com)
#### This dataset consists of reviews of Fine Foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.
#### Columns: Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text

#### Objective: Use RAG to generate sumnmaries of customer comments on specific topics

In [1]:
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer

# you can set envs using Colab secrets
dotenv.load_dotenv()

fireworks.client.api_key = 'KTGKcoCndQttxHOjG4cYALmEXR0ByhYBgtrozJesElA5eJ2A'

##### Define function to use an open-source LLM to do decoding. Retrieval-augmented generation (RAG) is a technique for enhancing the accuracy and reliability of generative AI models with facts fetched from external sources.

In [7]:
def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

#### RAG consists of a couple of major components: 1) an encoded knowledge base 2) any query text is then encoded and a search is done against the knowledgebase returning relevant contextual results 3) the results are then submitted to an LLM to create the final output

##### Please refer to separate file CreateCommentEmbeddings.py for encoding and creating vector embeddings of the summary and detailed comment fields. This is then stored in a ChromaDB collection. ChromaDB is a specialised datastore for storing, retrieving and searching for embeddings.

##### Tried a few different models (such as all-MiniLM-L6-v2) within Sentence Transformer but got best results (most relevant search results) from all-mpnet-base-v2

##### Now we try and query the ChromaDB collection to see the sorts of results we can get

In [5]:
collection = client.get_or_create_collection(
    name=f"amazon_ff_comments2",
    embedding_function=embed_fn
)

retriever_results = collection.query(
    query_texts=["gluten"],
    n_results=5,
)

print(retriever_results)

{'ids': [['1549', '6504', '3104', '4158', '1545']], 'distances': [[0.7743430137634277, 0.7746836543083191, 0.7987347841262817, 0.8155220746994019, 0.8267775774002075]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [["Gluten free goodness.Very tasty gluten-free option.  My kids loved it as it wasn't too spicy.", 'not gluten free.be careful...even though this is listed under gluten free section it is NOT gluten free', 'Not Gluten Free!!!!!!.Even though this item is listed under the Gluten free category, it is not.  The ingredients state that it contains wheat protein.  Do not order this if you need GF food.  Bad reviewing by Amazon!', 'Great gluten free product.As far as gluten free products this is awesome.  I use it as a substitute for any recipe that asks for flour.  It makes great pumpkin waffles.', 'Now containing gluten!.A year or so ago, I use to buy these quite often because they were gluten-free.  Last month I decided to order some again and th

##### Results look fairly relevant, I tried with a few different terms like "dog food", "coffee", etc.

##### Now we put together the rest of the solution where we take a search term, retrieve results from the collection (i.e. non-parametric values) and then pass it to an LLM (which has the parametric values) to get a final output of readable summary of results.

In [9]:
collection = client.get_or_create_collection(
    name=f"amazon_ff_comments2",
    embedding_function=embed_fn
)

# user query
query_text="dog food"
encoded_user_query= embedding_model1.encode(query_text)
user_query=encoded_user_query.tolist()

retriever_results = collection.query(
    query_embeddings=user_query,
    n_results=5,
)

# concatenate titles into a single string
results = '\n'.join(retriever_results['documents'][0])

prompt_template = f'''[INST]

Generate a summary of comments

Topic: {query_text}
Results: {results}

Comments:

[/INST]
'''
mistral_llm = "mistral-7b-instruct-4k"
responses = get_completion(prompt_template, mistral_llm, max_tokens=10000)
comment_summary = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Summarised comments:")
print(responses)
print("\n\n\nPrompt Template:")
print(prompt_template)

Summarised comments:

* The first commenter is happy with the new dog food they tried and wishes there were more coupons available for it.
* The second commenter tried the new dog food but found it to be unflavorful and not as easy to serve as they had hoped. They also criticized the lack of science behind the brand compared to higher-end brands like Royal Canine.
* The third commenter praises the new dog food as the only brand that their dog with allergies can eat.
* The fourth commenter mentions that the dogs on their Christmas list enjoyed the dog treats and ate them right away.
* The fifth commenter praises the new dog food as being healthy and good for digestion, as well as suitable for small puppies. They also mention that their dog eats their required amount at every feeding.



Prompt Template:
[INST]

Generate a summary of comments

Topic: dog food
Results: DOG FOOD.This was a new food for my dog, and he seems to have adjusted very well to this product. Thank you, and I wish t